###Ingestion de la carpeta "production_company" con archivos .csv

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

####Paso 1- Leer el archivo CSV usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
production_companies_schema = StructType(fields=[
    StructField("companyId", IntegerType(), True),
    StructField("companyName", StringType(), True)
])

In [0]:
production_companies_df = spark.read \
                .schema(production_companies_schema) \
                .csv(f"{bronze_folder_path}/{v_file_date}/production_company")
#                .csv("/mnt/mymoviehistory/bronze/production_company/production_company_*.csv")                

In [0]:
display(production_companies_df)

In [0]:
production_companies_df.count()
#para ver todos los registros que se han ingestado de todos los archivos

####Paso 2 - Renombrar las columnas y añadir nuevas columnas
- "companyId" renombar a "company_id", "companyName" a "Company_name"
- Agregar las columnas "ingestion_date" y "environment"

In [0]:
from pyspark.sql.functions import col, current_timestamp, lit

In [0]:
production_companies_final_df = add_ingestion_dat(production_companies_df) \
                            .withColumnsRenamed({"companyId":"company_id",
                                                  "companyName":"company_name"}) \
                            .withColumn("environment", lit(v_environment)) \
                            .withColumn("file_date",lit(v_file_date))
display(production_companies_final_df)

####Paso 3 - Escribir la salida en un formato "Parquet"


In [0]:
#production_companies_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/productions_companies")
#production_companies_final_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.productions_companies")
merge_condition='tgt.company_id = src.company_id and tgt.file_date = src.file_date'
merge_delta_lake(production_companies_final_df, "movie_silver", "productions_companies", silver_folder_path, merge_condition, "file_date")

In [0]:
production_companies_final_df.write.mode("append").partitionBy("file_date").format("delta").save(f"{silver_folder_path}/productions_companies")

In [0]:
%sql
select file_date, count(1) 
from movie_silver.productions_companies
group by file_date;

In [0]:
display(spark.read.format("delta").load("/mnt/mymoviehistory/silver/productions_companies"))

In [0]:
dbutils.notebook.exit("Success")